# Бэггинг

# Бэггинг над решающими деревьями

В этом блокноте мы увидим на примере, насколько позволяет улучшить качество по сравнению с одиночным деревом использование бэггинга и добавление в деревьях рандомизации при выборе признаков для разбиений.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression # для сравнения с линейной моделью

In [2]:
data = pd.read_csv('HR.csv')

target = 'left'
features = [c for c in data if c != target]
print(features)

X, y = data[features], data[target]

['last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']


In [8]:
d3 = DecisionTreeClassifier() # Обычное решающее дерево

Качество классификации решающим деревом с настройками по-умолчанию:

In [12]:
print("Decision tree:", cross_val_score(d3, X, y).mean())

Decision tree: 0.6536431686337267


Бэггинг над решающими деревьями:

In [13]:
print("D3 bagging:", cross_val_score(BaggingClassifier(d3, random_state=42), X, y).mean())

D3 bagging: 0.7174495299059812


Усредненная модель оказалась заметно лучше. У решающих деревьев есть существенный недостаток - нестабильность получаемого дерева при небольших изменениях в выборке. Но бэггинг обращает этот недостаток в достоинство, ведь усредненная модель работает лучше, когда базовые модели слабо скоррелированы (это обстоятельство будет пояснено дополнительно ниже - в теоретической части).

Изучив параметры DecisionTreeClassifier, можно найти хороший способ сделать деревья еще более различными - при построении каждого узла отбирать случайные max_features признаков и искать информативное разбиение только по одному из них.

In [ ]:
# Решающее дерево с рандомизацией в сплитах
rnd_d3 = DecisionTreeClassifier(max_features=int(len(features) ** 0.5))

In [ ]:
?DecisionTreeClassifier

In [14]:
print("Randomized D3 Bagging:", cross_val_score(BaggingClassifier(rnd_d3, random_state=42), X, y).mean())

Randomized D3 Bagging: 0.7194494632259785


В среднем, качество получается еще лучше. Для выбора числа признаков использовалась часто применяемая на практике эвристика - брать корень из общего числа признаков. Если бы мы решали задачу регрессии - брали бы треть от общего числа.

In [15]:
print("Random Forest:", cross_val_score(RandomForestClassifier(random_state=42), X, y).mean())

Random Forest: 0.7232495965859839


Кстати, давайте посмотрим, нужно ли выбирать случайные признаки в каждом сплите или достаточно выбрать их один раз для каждого дерева. В этом нам поможет параметр max_features в BaggingClassifier:

In [5]:
?BaggingClassifier

In [9]:
print("D3 bagging:", cross_val_score(BaggingClassifier(d3, random_state=42, max_features=int(len(features) ** 0.5)), X, y).mean())

D3 bagging: 0.7175827298793092


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [16]:
print("Logistic Regression:", cross_val_score(LogisticRegression(), X, y).mean())

Logistic Regression: 0.6287053143962126


## Bonus-track

#### Опциональное практическое задание
Повторные запуски cross_val_score будут показывать различное качество модели.

Это зависит от параметра рандомизации модели "random_state" в DecisionTreeClassifier, BaggingClassifie или RandomForest.

Чтобы определить, действительно ли одна модель лучше другой, можно посмотреть на её качество в среднем, то есть усредняя запуски с разным random_state. Попробуйте сравнить качество и понять, действительно ли BaggingClassifier(d3) лучше BaggingClassifier(rnd_d3).

Также попробуйте подумать, чем здесь отличается BaggingClassifier(rnd_d3) от RandomForestClassifier()?

#### Немного теории: почему усреднение увеличивает "устойчивость" модели

Пусть есть случайные одинаково распределённые величины $\xi_1, \xi_2, \dots, \xi_n$, скоррелированные с коэффициентом корреляции $\rho$ и дисперсией $\sigma^2$. Какова будет дисперсия величины $\frac1n \sum_{i=1}^n \xi_i$?

$$\mathbf{D} \frac1n \sum_{i=1}^n \xi_i = \frac1{n^2}\mathbf{cov} (\sum_{i=1}^n \xi_i, \sum_{i=1}^n \xi_i) = \frac1{n^2} \sum_{i=1, j=1}^n \mathbf{cov}(\xi_i, \xi_j) = \frac1{n^2} \sum_{i=1}^n \mathbf{cov}(\xi_i, \xi_i) + \frac1{n^2} \sum_{i=1, j=1, i\neq j}^n \mathbf{cov}(\xi_i, \xi_j) = \frac1{n^2} \sum_{i=1}^n \sigma^2+ \frac1{n^2} \sum_{i=1, j=1, i\neq j}^n \rho \sigma^2 =$$
$$ = \frac1{n^2} n \sigma^2 + \frac1{n^2} n(n-1) \rho \sigma^2  = \frac{\sigma^2( 1 + \rho(n-1))}{n}$$

Таким образом, чем менее величины скоррелированы между собой, тем меньше будет дисперсия после их усреднения. В этом и состоит усреднения прогнозов моделей: давайте сделаем много моделей, желательно с не очень большой корреляцией ответов, а потом усредим их прогнозы, тогда предсказание станет более устойчивым, чем у отдельных моделей.

**Опциональные теоретические задачи**

1.

Покажите, что матожидание квадрата ошибки модели, полученной усреднением $M$ регрессионных моделей $a_i(x)$, ответы коротых не скоррелированы, а матожидание $\mathbf{E}_{x,y}\left( a_i(x) - y \right)$ равно нулю для каждой модели, будет в $M$ раз меньше, чем матожидание квадрата ошибки каждой модели $a_i(x)$

2.

Покажите, что средний квадрат отклонения от правильных ответов на обучающей выборке (**MSE**, mean squared error) у усредненной модели будет всегда не больше, чем среднее значение **MSE** на обучающей выборке по всем усредняемым моделям. *Подсказка: вам помогут неравенство Коши-Буняковского (оно же неравенство Шварца) или неравенство Йенсена.*

3.

Обобщите результат задачи 2 для любой выпуклой функции потерь.